![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [13]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

In [14]:
data = pd.read_csv('car_insurance.csv')

data.info()

print(data['postal_code'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [15]:
# Convert columns to types compatible with .corr()
data['age'] = data['age'].astype(float)
data['gender'] = data['gender'].astype('category').cat.codes
data['driving_experience'] = data['driving_experience'].astype('category').cat.codes
data['education'] = data['education'].astype('category').cat.codes
data['income'] = data['income'].astype('category').cat.codes
data['credit_score'] = data['credit_score'].astype(float)
data['vehicle_ownership'] = data['vehicle_ownership'].astype(int)
data['vehicle_year'] = data['vehicle_year'].astype('category').cat.codes
data['married'] = data['married'].astype(int)
data['children'] = data['children'].astype(float)
data['annual_mileage'] = data['annual_mileage'].astype(float)
data['vehicle_type'] = data['vehicle_type'].astype('category').cat.codes
data['speeding_violations'] = data['speeding_violations'].astype(float)
data['duis'] = data['duis'].astype(float)
data['past_accidents'] = data['past_accidents'].astype(float)
data['outcome'] = data['outcome'].astype(int)

data["credit_score"].fillna(data["credit_score"].mean(), inplace=True)
data["annual_mileage"].fillna(data["annual_mileage"].mean(), inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   gender               10000 non-null  int8   
 3   driving_experience   10000 non-null  int8   
 4   education            10000 non-null  int8   
 5   income               10000 non-null  int8   
 6   credit_score         10000 non-null  float64
 7   vehicle_ownership    10000 non-null  int64  
 8   vehicle_year         10000 non-null  int8   
 9   married              10000 non-null  int64  
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       10000 non-null  float64
 13  vehicle_type         10000 non-null  int8   
 14  speeding_violations  10000 non-null  float64
 15  duis                 10000 non-null  

In [16]:
cols = list(data.columns)

cols_object = list(data.select_dtypes(include='object').columns)
cols_object.extend(['outcome', 'id'])

cols = set(cols) - set(cols_object)

print(cols)

{'vehicle_year', 'children', 'vehicle_type', 'past_accidents', 'education', 'annual_mileage', 'age', 'postal_code', 'income', 'duis', 'vehicle_ownership', 'married', 'driving_experience', 'gender', 'credit_score', 'speeding_violations'}


In [17]:
models = {}
for col in cols:
    models[col] = logit(f"outcome ~ {col}", data=data).fit()

accuracies = {}
for key, model in models.items():
    # print(key,model)
    conf_matrix = model.pred_table()
    tn = conf_matrix[0,0]
    tp = conf_matrix[1,1]
    fn = conf_matrix[1,0]
    fp = conf_matrix[0,1]
    
    acc = (tn + tp) / (tn + fn + fp + tp)
    accuracies[key] = acc
    
accuracy_df = pd.Series(accuracies)

print(accuracy_df.idxmax())
accuracy_df.sort_values(ascending=False, inplace=True)
print(accuracy_df)

best_feature_df = {'best_feature': accuracy_df.idxmax(), 'best_accuracy': max(accuracy_df)}
best_feature_df = pd.Series(best_feature_df).to_frame().T
print(best_feature_df, type(best_feature_df))


Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.595431
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621700
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.549220
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.617408
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.605716
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.617345
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.620588
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.598699
  